In [3]:
%matplotlib inline
import matplotlib.image as mpimg
import os
from glob import iglob
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from skimage.feature import hog
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label
import math
from collections import defaultdict

In [4]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 255)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        if(file.endswith('.png')):
            image = (image * 255).astype(np.uint8)
            
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy



# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows


In [5]:
cars = []
not_cars = []

car_images = 'vehicles/vehicles/**/*.png'
noncar_images = 'non-vehicles/non-vehicles/**/*.png'

for file in iglob(car_images, recursive=True):
    cars.append(file)

for file in iglob(noncar_images, recursive=True):
    not_cars.append(file)

print(len(cars), len(not_cars))

8792 8968


In [6]:
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 64    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, 700] # Min and max in y to search in slide_window()

In [10]:
# def train():
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(not_cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# svc = SVC(kernel = 'rbf')
# svc = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_features = 'auto')
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()


/home/prudhvi/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6252
10.73 Seconds to train SVC...
Test Accuracy of SVC =  0.9921


In [7]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def area(bbox):
    return abs((bbox[0][0] - bbox[1][0]) * (bbox[0][1] - bbox[1][1]))

bboxes = []
bboxes_list = []

def draw_labeled_bboxes(img, labels):
    maxArea = defaultdict(lambda: ((0,0),(0,0)))
    
    bbox_temp = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
#         cv2.rectangle(img, bbox[0], bbox[1], (0,255,0), 8)
        bbox_temp.append(bbox)
    
        for (ind,(box, count)) in enumerate(bboxes):
            mid = [(box[0][0] + box[1][0])/2, (box[0][1] + box[1][1])/2]
            bmid = [(bbox[0][0] + bbox[1][0])/2, (bbox[0][1] + bbox[1][1])/2]
#             print(mid, bmid)
            if abs(mid[0] - bmid[0]) + abs(mid[1] - bmid[1]) < 300:
                if area(bbox) > area(maxArea[ind]):
                    maxArea[ind] = bbox
                bboxes[ind][1] += 1
                break
        else:
            maxArea[len(bboxes)] = bbox    
            bboxes.append([bbox,1])
    
    bboxes_list.append(bbox_temp)
    
    for (ind,(box, count)) in enumerate(bboxes):
        if area(maxArea[ind]) is 0 and bboxes[ind][1] >= 2.0:
            bboxes[ind][1] -= 2.
    
    print(bboxes)
    for (ind, (bbox, count)) in enumerate(bboxes):
        fbox = maxArea[ind] if area(maxArea[ind]) > area(bbox) else bbox
        if (area(maxArea[ind]) > bboxes[ind][1]) or (area(maxArea[ind]) > 2500):
            bboxes[ind][0] = maxArea[ind]
#         area check for noise reduction if any
        if (count >= 12) and (area(fbox) > 2000) and (fbox[0][0] >= 500):
            cv2.rectangle(img, fbox[0], fbox[1], (255,0,0), 8)
#         elif count > 0.3:
#             bboxes[ind][1] = float(count) - 0.3
    # Return the image
    return img



def draw(image):
    image = np.array(image)
    draw_image = np.copy(image)
    
    if not hasattr(draw, "windows"):
        draw.windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(64, 64), xy_overlap=(0.7, 0.7))
        draw.windows += slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(32, 32), xy_overlap=(0.7, 0.7))
    
    hot_windows = search_windows(image, draw.windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       
    
#     window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    
#     return window_img
    
    
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat, hot_windows)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat, 1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    
    return draw_img
    
from sklearn.externals import joblib
def loadclf(path):
    return joblib.load(path)

def saveclf(clf, path):
    joblib.dump(clf, path)
    
def savescaler(scaler,path):
    joblib.dump(scaler, path)
def loadscaler(path):
    return joblib.load(path)

In [ ]:
for file in iglob('test_images/*.jpg'):
    img = mpimg.imread(file)
    img = np.array(img)
    plt.figure()
    plt.imshow(draw(img))

In [8]:
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    return img

def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, conv='HLS')
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]
#     print(ch1,ch2,ch3)
    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
#             Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
#             hog_features = np.hstack((hog1, hog2, hog3))
#             print(len(hog_features))
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            if spatial_feat:
                spatial_features = bin_spatial(subimg, size=spatial_size)
            else:
                spatial_features = []
                
            if hist_feat:
                hist_features = color_hist(subimg, nbins=hist_bins)
            else:
                hist_features = []

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
#             test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                
    return draw_img

scale = 1.0


for file in iglob('test_images/*.jpg'):
    img = mpimg.imread(file)
    img = np.array(img)
    plt.figure()
    out_img = find_cars(img, y_start_stop[0], y_start_stop[1], scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    plt.imshow(out_img)


NameError: name 'svc' is not defined

In [15]:
saveclf(svc, 'model.pkl')
savescaler(X_scaler, "scaler.pkl")

In [9]:
svc = loadclf('model.pkl')
X_scaler = loadscaler("scaler.pkl")

In [48]:
import pickle
# pickle.dump(bboxes_list,  open( "boxes.p", "wb" ))
def non_max_suppression_fast(boxes, overlapThresh):
   # if there are no boxes, return an empty list
   if len(boxes) == 0:
      return []

   # if the bounding boxes integers, convert them to floats --
   # this is important since we'll be doing a bunch of divisions
   if boxes.dtype.kind == "i":
      boxes = boxes.astype("float")
#  
   # initialize the list of picked indexes   
   pick = []

   # grab the coordinates of the bounding boxes
   x1 = boxes[:,0]
   y1 = boxes[:,1]
   x2 = boxes[:,2]
   y2 = boxes[:,3]

   # compute the area of the bounding boxes and sort the bounding
   # boxes by the bottom-right y-coordinate of the bounding box
   area = (x2 - x1 + 1) * (y2 - y1 + 1)
   idxs = np.argsort(y2)

   # keep looping while some indexes still remain in the indexes
   # list
   while len(idxs) > 0:
      # grab the last index in the indexes list and add the
      # index value to the list of picked indexes
      last = len(idxs) - 1
      i = idxs[last]
      pick.append(i)

      # find the largest (x, y) coordinates for the start of
      # the bounding box and the smallest (x, y) coordinates
      # for the end of the bounding box
      xx1 = np.maximum(x1[i], x1[idxs[:last]])
      yy1 = np.maximum(y1[i], y1[idxs[:last]])
      xx2 = np.minimum(x2[i], x2[idxs[:last]])
      yy2 = np.minimum(y2[i], y2[idxs[:last]])

      # compute the width and height of the bounding box
      w = np.maximum(0, xx2 - xx1 + 1)
      h = np.maximum(0, yy2 - yy1 + 1)

      # compute the ratio of overlap
      overlap = (w * h) / area[idxs[:last]]

      # delete all indexes from the index list that have
      idxs = np.delete(idxs, np.concatenate(([last],
         np.where(overlap > overlapThresh)[0])))

   # return only the bounding boxes that were picked using the
   # integer data type
   return boxes[pick].astype("int")

In [51]:
list_boxes = pickle.load(open("boxes.p", "rb"))
index = 0
bboxes = []

def new_draw(img):
    global index
    maxArea = defaultdict(lambda: ((0,0),(0,0)))
    # Iterate through all detected cars
    for bbox in list_boxes[index]:

        for (ind,(box, count)) in enumerate(bboxes):
            mid = [(box[0][0] + box[1][0])/2, (box[0][1] + box[1][1])/2]
            bmid = [(bbox[0][0] + bbox[1][0])/2, (bbox[0][1] + bbox[1][1])/2]
#             print(mid, bmid)
            if abs(mid[0] - bmid[0]) + abs(mid[1] - bmid[1]) < 150:
                if area(bbox) > area(maxArea[ind]):
                    maxArea[ind] = bbox
                bboxes[ind][1] += 1
                break
        else:
            maxArea[len(bboxes)] = bbox    
            bboxes.append([bbox,1])

    for (ind,(box, count)) in enumerate(bboxes):
        if area(maxArea[ind]) is 0 and bboxes[ind][1] >= 2.0:
            bboxes[ind][1] -= 2.

    print(bboxes)
    for (ind, (bbox, count)) in enumerate(bboxes):
#         fbox = maxArea[ind] if area(maxArea[ind]) > area(bbox) else bbox
        rects = np.array([[maxArea[ind][0][0], maxArea[ind][0][1], maxArea[ind][1][0], maxArea[ind][1][1]],
                 [bbox[0][0], bbox[0][1], bbox[1][0], bbox[1][1]]], dtype=np.int32)
        fboxes = non_max_suppression_fast(rects, 10000)
        if len(fboxes) == 4:
            fbox = [[fboxes[0][0], fboxes[0][1]], [fboxes[0][2], fboxes[0][3]]]
        else:
            fbox = bbox
        if (area(maxArea[ind]) > bboxes[ind][1]) or (area(maxArea[ind]) > 3500):
            bboxes[ind][0] = maxArea[ind]
#         area check for noise reduction if any
        if (count >= 25) and (area(fbox) > 2500) and (fbox[0][0] >= 600):
            cv2.rectangle(img, fbox[0], fbox[1], (255,0,0), 8)
    # Return the image
    index += 1
    return img

In [52]:
from moviepy.editor import VideoFileClip, CompositeVideoClip
from IPython.display import HTML

output = 'out.mp4'
clip = VideoFileClip("project_video.mp4").subclip(5,)
detected   = clip.fl_image(new_draw)
detected.write_videofile(output, audio=False)

[[((234, 418), (274, 449)), 1]]
[MoviePy] >>>> Building video out.mp4
[MoviePy] Writing video out.mp4





  0%|          | 0/1136 [00:00<?, ?it/s]

[[((234, 418), (274, 449)), 2]]
[[((234, 418), (274, 449)), 3], [((36, 427), (58, 458)), 1]]





  0%|          | 2/1136 [00:02<19:28,  1.03s/it]

[[((225, 418), (256, 449)), 4], [((36, 427), (58, 458)), 2], [((1224, 463), (1255, 494)), 1]]
[[((225, 418), (238, 449)), 2.0], [((9, 427), (40, 449)), 0.0], [((1224, 463), (1255, 494)), 1]]
[[((225, 418), (238, 449)), 0.0], [((9, 427), (40, 449)), 0.0], [((1224, 463), (1255, 494)), 2]]
[[((225, 418), (238, 449)), 2.0], [((9, 427), (40, 449)), 0.0], [((1215, 481), (1219, 503)), 0.0]]


...

[[((9, 418), (31, 431)), 54.0], [((798, 400), (877, 482)), 683.0], [((1224, 438), (1273, 476)), 1518.0], [((756, 607), (778, 620)), 0.0], [((243, 418), (274, 440)), 1.0], [((891, 571), (904, 584)), 1.0], [((1179, 616), (1210, 620)), 0.0], [((585, 625), (616, 647)), 0.0], [((522, 607), (553, 629)), 1.0], [((1233, 652), (1237, 665)), 0.0], [((306, 409), (310, 440)), 0.0], [((441, 634), (472, 656)), 1.0], [((756, 517), (804, 558)), 0.0], [((963, 508), (985, 539)), 1.0], [((27, 553), (31, 584)), 0.0], [((135, 499), (139, 530)), 0.0], [((405, 400), (418, 431)), 0.0], [((603, 625), (634, 656)), 1.0], [((1215, 427), (1260, 458)), 0.0], [((333, 418), (364, 440)), 0.0], [((189, 490), (220, 494)), 1.0], [((756, 571), (787, 602)), 0.0], [((1224, 652), (1246, 683)), 0.0], [((1053, 409), (1057, 440)), 1.0], [((909, 427), (956, 463)), 0.0], [((837, 616), (868, 638)), 0.0], [((266, 661), (292, 672)), 1.0], [((63, 625), (103, 665)), 0.0], [((396, 454), (427, 485)), 1], [((576, 400), (598, 431)), 1], [




 99%|█████████▉| 1123/1136 [04:09<00:00, 41.16it/s]


 99%|█████████▉| 1129/1136 [04:09<00:00, 45.27it/s]

[[((27, 499), (58, 521)), 34.0], [((783, 400), (868, 482)), 696.0], [((1224, 438), (1273, 476)), 1492.0], [((756, 607), (778, 620)), 0.0], [((180, 409), (211, 431)), 2.0], [((891, 625), (922, 647)), 0.0], [((1179, 616), (1210, 620)), 0.0], [((585, 625), (616, 647)), 0.0], [((522, 607), (553, 629)), 1.0], [((1233, 652), (1237, 665)), 0.0], [((306, 409), (310, 440)), 0.0], [((441, 634), (472, 656)), 1.0], [((756, 517), (804, 558)), 0.0], [((963, 508), (985, 539)), 1.0], [((72, 508), (112, 548)), 1.0], [((135, 499), (139, 530)), 0.0], [((405, 400), (418, 431)), 0.0], [((603, 625), (634, 656)), 1.0], [((1215, 427), (1260, 458)), 0.0], [((333, 418), (364, 440)), 0.0], [((189, 490), (220, 494)), 1.0], [((756, 571), (787, 602)), 0.0], [((1224, 652), (1246, 683)), 0.0], [((1053, 409), (1057, 440)), 1.0], [((909, 427), (956, 463)), 0.0], [((837, 616), (868, 638)), 0.0], [((266, 661), (292, 672)), 1.0], [((63, 625), (103, 665)), 0.0], [((396, 454), (427, 485)), 1], [((576, 400), (598, 431)), 1],




100%|█████████▉| 1135/1136 [04:10<00:00, 45.13it/s]




[[((9, 418), (49, 449)), 17.0], [((774, 409), (850, 482)), 711.0], [((1170, 400), (1264, 482)), 1497.0], [((693, 634), (697, 656)), 0.0], [((108, 418), (139, 449)), 1.0], [((891, 625), (922, 647)), 0.0], [((1179, 616), (1210, 620)), 0.0], [((486, 571), (517, 602)), 0.0], [((486, 571), (508, 602)), 0.0], [((1233, 652), (1237, 665)), 0.0], [((306, 409), (310, 440)), 0.0], [((441, 634), (472, 656)), 1.0], [((648, 499), (679, 521)), 0.0], [((963, 508), (985, 539)), 1.0], [((72, 508), (112, 548)), 1.0], [((135, 499), (139, 530)), 0.0], [((405, 400), (418, 431)), 0.0], [((603, 625), (634, 656)), 1.0], [((1215, 427), (1260, 458)), 0.0], [((333, 418), (364, 440)), 0.0], [((189, 490), (220, 494)), 1.0], [((756, 571), (787, 602)), 0.0], [((1224, 652), (1246, 683)), 0.0], [((1053, 409), (1057, 440)), 1.0], [((909, 427), (956, 463)), 0.0], [((837, 616), (868, 638)), 0.0], [((266, 661), (292, 672)), 1.0], [((63, 625), (103, 665)), 0.0], [((396, 454), (427, 485)), 1], [((576, 400), (598, 431)), 1], 

In [54]:
HTML("""
<video width="640" controls>
  <source src="{0}">
</video>"
""".format(output))